In [21]:
import pandas as pd
import numpy as np
from IPython.display import display

# Load datasets
q152 = pd.read_csv("q152_country_year_top2_labels.csv")
q154 = pd.read_csv("q154_country_year_top2_labels.csv")

# Merge on country and year, keep only rows where has_second_label==True in both
merged = pd.merge(
    q152[q152['has_second_label'] == True],
    q154[q154['has_second_label'] == True],
    on=['B_COUNTRY_ALPHA', 'A_YEAR'],
    suffixes=('_152', '_154')
)

# Function to extract question and likert scale from label string (e.g. 'Q152_2')
def extract_q_likert(label):
    if pd.isna(label):
        return (None, np.nan)
    parts = label.split('_')
    if len(parts) != 2:
        return (None, np.nan)
    return parts[0], int(parts[1])

# Extract qid and likert scale for all relevant columns
for col in ['most_frequent_label_152', 'second_most_frequent_label_152',
            'most_frequent_label_154', 'second_most_frequent_label_154']:
    merged[[f'{col}_qid', f'{col}_likert']] = merged[col].apply(lambda x: pd.Series(extract_q_likert(x)))

# Map likert scales for each question
merged['Q152_likert'] = merged['most_frequent_label_152_likert']
merged['Q153_likert'] = merged['second_most_frequent_label_152_likert']
merged['Q154_likert'] = merged['most_frequent_label_154_likert']
merged['Q155_likert'] = merged['second_most_frequent_label_154_likert']

# Voice component functions without coefficients
def compute_I_VOICE1(q154, q155):
    if pd.isna(q154) or pd.isna(q155):
        return np.nan
    if (q154 == 2 and q155 == 4) or (q154 == 4 and q155 == 2):
        return 1
    if (q154 == 2 and q155 != 4) or (q154 == 4 and q155 != 2):
        return 0.66
    if (q154 != 2 and q155 == 4) or (q154 != 4 and q155 == 2):
        return 0.33
    if q154 > -1 and q155 > -1:
        return 0
    return np.nan

def compute_I_VOICE2(q152, q153):
    if pd.isna(q152) or pd.isna(q153):
        return np.nan
    if q152 == 3:
        return 1
    if q153 == 3:
        return 0.5
    if q152 > -1 and q153 > -1:
        return 0
    return np.nan

# Calculate raw VOICE components, skipping metrics if NA
merged['I_VOICE1'] = merged.apply(lambda r: compute_I_VOICE1(r['Q154_likert'], r['Q155_likert']), axis=1)
merged['I_VOICE2'] = merged.apply(lambda r: compute_I_VOICE2(r['Q152_likert'], r['Q153_likert']), axis=1)

# Multiply by corresponding counts (weights)
merged['weighted_I_VOICE1'] = merged['I_VOICE1'] * (
    merged['most_frequent_count_154'].fillna(0) + merged['second_most_frequent_count_154'].fillna(0)
)
merged['weighted_I_VOICE2'] = merged['I_VOICE2'] * (
    merged['most_frequent_count_152'].fillna(0) + merged['second_most_frequent_count_152'].fillna(0)
)

# Sum counts for normalization, skipping counts where voice component is nan
merged['total_counts'] = (
    (merged['most_frequent_count_152'].fillna(0) + merged['second_most_frequent_count_152'].fillna(0)) * (~merged['I_VOICE2'].isna()).astype(int) +
    (merged['most_frequent_count_154'].fillna(0) + merged['second_most_frequent_count_154'].fillna(0)) * (~merged['I_VOICE1'].isna()).astype(int)
)

# Compute weighted average VOICE score
merged['VOICE_weighted'] = (
    (merged['weighted_I_VOICE1'].fillna(0) + merged['weighted_I_VOICE2'].fillna(0)) / merged['total_counts'].replace(0, np.nan)
)

# Load full WVS dataset with survey weights
wvs_full_path = r"C:\Users\secki\OneDrive\Desktop\MY498 Capstone Under Supervision\WVS Wave 7 Full Package for EFA\F00010736-WVS_Cross-National_Wave_7_rdata_v6_0\wvs7_full_data.csv"
wvs_full = pd.read_csv(wvs_full_path)

# Select only needed columns and drop duplicates
wvs_weights = wvs_full[['B_COUNTRY_ALPHA', 'A_YEAR', 'S018']].drop_duplicates()

# Merge survey weights into merged dataframe
merged = pd.merge(
    merged,
    wvs_weights,
    on=['B_COUNTRY_ALPHA', 'A_YEAR'],
    how='left'
)

# Calculate VOICE weighted by survey weights (handle missing survey weights by skipping multiplication)
merged['VOICE_weighted_survey'] = merged.apply(
    lambda r: r['VOICE_weighted'] * r['S018'] if not pd.isna(r['VOICE_weighted']) and not pd.isna(r['S018']) else r['VOICE_weighted'],
    axis=1
)

# Prepare final output dataframe and sort descending
voice_df_weighted = merged[['B_COUNTRY_ALPHA', 'A_YEAR', 'VOICE_weighted_survey']].copy()
voice_df_weighted = voice_df_weighted.rename(columns={'VOICE_weighted_survey': 'VOICE'})
voice_df_weighted = voice_df_weighted.sort_values(by='VOICE', ascending=False).reset_index(drop=True)

# Display final results
display(voice_df_weighted)



C:\Users\secki\AppData\Local\Temp\ipykernel_36848\2972118543.py:87: DtypeWarning: Columns (346,348,350,580,581,582,583) have mixed types. Specify dtype option on import or set low_memory=False.
  wvs_full = pd.read_csv(wvs_full_path)


,B_COUNTRY_ALPHA,A_YEAR,VOICE
0,ARG,2017,0.634460
1,SRB,2017,0.621891
2,LBY,2022,0.613155
3,VEN,2021,0.588235
4,LBN,2018,0.502941
5,MMR,2020,0.500617
6,AND,2018,0.473307
7,JPN,2019,0.448386
8,IRN,2020,0.431660
9,URY,2022,0.416842


In [22]:
import pandas as pd
from scipy.stats import pearsonr

# Load your sparse voice_df from previous steps (make sure voice_df is available)
# voice_df = ...  # Already available from previous processing

# Load full WVS country-year voice dataset
wvs_full_path = r"C:\Users\secki\OneDrive\Desktop\MY498 Capstone Under Supervision\WVS Wave 7 Full Package for EFA\F00010736-WVS_Cross-National_Wave_7_rdata_v6_0\wvs_countryyear_full_components.csv"
wvs_df = pd.read_csv(wvs_full_path)

# Inspect column names in wvs_df to find the VOICE variable and country/year column names
print(wvs_df.columns)

# Suppose country and year columns are 'country' and 'year' and voice column is 'VOICE' (adjust if different)
# Rename columns in wvs_df to match voice_df if needed
wvs_df.rename(columns={'country': 'B_COUNTRY_ALPHA', 'year': 'A_YEAR'}, inplace=True)

# Merge the two datasets on country and year, inner join to keep only common observations
merged_df = pd.merge(
    voice_df_weighted,
    wvs_df[['B_COUNTRY_ALPHA', 'A_YEAR', 'VOICE']],
    on=['B_COUNTRY_ALPHA', 'A_YEAR'],
    how='inner',
    suffixes=('_calc', '_wvs')
)

print(f"Merged dataset size: {len(merged_df)}")

# Calculate Pearson correlation between the two VOICE measures
corr, p_value = pearsonr(merged_df['VOICE_calc'], merged_df['VOICE_wvs'])

print(f"Pearson correlation between calculated VOICE and WVS VOICE: {corr:.4f} (p-value = {p_value:.4g})")


Index(['S025', 'B_COUNTRY', 'B_COUNTRY_ALPHA', 'A_YEAR', 'SCEPTICISM',
       'RELATIVISM', 'DISBELIEF', 'DEFIANCE', 'SACSECVAL', 'AUTONOMY',
       'EQUALITY', 'CHOICE', 'VOICE', 'RESEMAVAL'],
      dtype='object')
Merged dataset size: 58
Pearson correlation between calculated VOICE and WVS VOICE: -0.0746 (p-value = 0.5776)
